# Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download dataset

In [ ]:
import gdown
url = '1-CNx_k1yEJ-RDC94nXUMlhtc1FQH7lzt'
gdown.download(id=url, output="test_set.tar.gz")

In [5]:
!mkdir -p test

!tar -zxf '/content/test_set.tar.gz' -C /content/test
!rm '/content/test_set.tar.gz'

test_path = '/content/test'

# Install dependencies and libraries

In [ ]:
# Install dependencies
!pip install lief==0.12.0
!pip install numpy
!pip install deap
!pip install pandas
!pip install matplotlib
!pip install tqdm
!pip install python-magic
# Install ML-Pentest Lib
!pip install ml-pentest

In [7]:
from ml_pentest.attacks.blackbox.genetic_attack.GAMMA.gamma_section_injection import GammaSectionInjection
from ml_pentest.attacks.blackbox.genetic_attack.GAMMA.attack_utils import create_section_population_from_folder
from ml_pentest.models.wrappers.malconv2_wrapper import MalConvWrapper

import os
import torch
import lief
import numpy as np
import random

# Models definition

In [8]:
"""
Classifying Sequences of Extreme Length with Constant Memory Applied to Malware Detection
Edward Raff, William Fleshman, Richard Zak, Hyrum Anderson and Bobby Filar and Mark Mclean
https://arxiv.org/abs/2012.09390
"""
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

def drop_zeros_hook(module, grad_input, grad_out):
    """
    This function is used to replace gradients that are all zeros with None
    In pyTorch None will not get back-propogated
    So we use this as a approximation to saprse BP to avoid redundant and useless work
    """
    grads = []
    with torch.no_grad():
        for g in grad_input:
            if torch.nonzero(g).shape[0] == 0:
                grads.append(g.to_sparse())
            else:
                grads.append(g)

    return tuple(grads)


class CatMod(torch.nn.Module):
    def __init__(self):
        super(CatMod, self).__init__()

    def forward(self, x):
        return torch.cat(x, dim=2)


class LowMemConvBase(nn.Module):

    def __init__(self, chunk_size=65536, overlap=512, min_chunk_size=1024):
        """
        chunk_size: how many bytes at a time to process. Increasing may improve compute efficent, but use more memory. Total memory use will be a function of chunk_size, and not of the length of the input sequence L

        overlap: how many bytes of overlap to use between chunks

        """
        super(LowMemConvBase, self).__init__()
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.min_chunk_size = min_chunk_size

        #Used for pooling over time in a more efficent way
        self.pooling = nn.AdaptiveMaxPool1d(1)
        self.cat = CatMod()
        self.cat.register_backward_hook(drop_zeros_hook)
        self.receptive_field = None

        #Used to force checkpoint code to behave correctly due to poor design https://discuss.pytorch.org/t/checkpoint-with-no-grad-requiring-inputs-problem/19117/11
        self.dummy_tensor = torch.ones(1, dtype=torch.float32, requires_grad=True)

    def processRange(self, x, **kwargs):
        """
        This method does the work to convert an LongTensor input x of shape (B, L) , where B is the batch size and L is the length of the input. The output of this functoin should be a tensor of (B, C, L), where C is the number of channels, and L is again the input length (though its OK if it got a little shorter due to convs without padding or something).

        """
        pass

    def determinRF(self):
        """
        This function evaluates the receptive field & stride of our sub-network.
        """

        if self.receptive_field is not None:
            return self.receptive_field, self.stride, self.out_channels

        if not hasattr(self, "device_ids"):
            #We are training with just one device. Lets find out where we should move the data
            cur_device = next(self.embd.parameters()).device
        else:
            cur_device = "cpu"

        #Lets do a simple binary search to figure out how large our RF is.
        #It can't be larger than our chunk size! So use that as upper bound
        min_rf = 1
        max_rf = self.chunk_size

        with torch.no_grad():

            tmp = torch.zeros((1,max_rf)).long().to(cur_device)

            while True:
                test_size = (min_rf+max_rf)//2
                is_valid = True
                try:
                    self.processRange(tmp[:,0:test_size])
                except:
                    is_valid = False

                if is_valid:
                    max_rf = test_size
                else:
                    min_rf = test_size+1

                if max_rf == min_rf:
                    self.receptive_field = min_rf
                    out_shape = self.processRange(tmp).shape
                    self.stride = self.chunk_size//out_shape[2]
                    self.out_channels = out_shape[1]
                    break


        return self.receptive_field, self.stride, self.out_channels


    def pool_group(self, *args):
        x = self.cat(args)
        x = self.pooling(x)
        return x

    def seq2fix(self, x, pr_args={}):
        """
        Takes in an input LongTensor of (B, L) that will be converted to a fixed length representation (B, C),
        where C is the number of channels provided by the base_network given at construction.
        """

        receptive_window, stride, out_channels = self.determinRF()

        if x.shape[1] < receptive_window: #This is a tiny input! Pad it out
            x = F.pad(x, (0, receptive_window-x.shape[1]), value=0) # 0 is the pad value
        batch_size = x.shape[0]
        length = x.shape[1]

        #Let's go through the input data without gradients first, and find the positions that "win"
        #the max-pooling. Most of the gradients will be zero, and we don't want to waste valuable
        #memory and time computing them.
        #Once we know the winners, we will go back and compute the forward activations on JUST
        #the subset of positions that won!
        winner_values = np.zeros((batch_size, out_channels))-1.0
        winner_indices = np.zeros((batch_size, out_channels), dtype=np.int64)

        if not hasattr(self, "device_ids"):
            #We are training with just one device. Lets find out where we should move the data
            cur_device = next(self.embd.parameters()).device
        else:
            cur_device = None

        step = self.chunk_size #- self.overlap
        start = 0
        end = start+step

        with torch.no_grad():
            while start < end and (end-start) >= max(self.min_chunk_size, receptive_window):
                x_sub = x[:,start:end]
                if cur_device is not None:
                    x_sub = x_sub.to(cur_device)
                activs = self.processRange(x_sub.long(), **pr_args)
                activ_win, activ_indx = F.max_pool1d(activs, kernel_size=activs.shape[2], return_indices=True)
                #We want to remove only last dimension, but if batch size is 1, np.squeeze
                #will screw us up and remove first dim too.
                #activ_win = np.squeeze(activ_win.cpu().numpy())
                #activ_indx = np.squeeze(activ_indx.cpu().numpy())
                activ_win = activ_win.cpu().numpy()[:,:,0]
                activ_indx = activ_indx.cpu().numpy()[:,:,0]
                selected = winner_values < activ_win
                winner_indices[selected] = activ_indx[selected]*stride + start
                winner_values[selected]  = activ_win[selected]
                start = end
                end = min(start+step, length)

        # Now we know every index that won, we need to compute values and with gradients!

        # Find unique winners for every batch
        final_indices = [np.unique(winner_indices[b,:]) for b in range(batch_size)]

        # Collect inputs that won for each batch
        chunk_list = [[x[b:b+1,max(i-receptive_window,0):min(i+receptive_window,length)] for i in final_indices[b]] for b in range(batch_size)]
        # Convert to a torch tensor of the bytes
        chunk_list = [torch.cat(c, dim=1)[0,:] for c in chunk_list]

        # Pad out shorter sequences to the longest one
        x_selected = torch.nn.utils.rnn.pad_sequence(chunk_list, batch_first=True)

        # Shape is not (B, L). Compute it.
        if cur_device is not None:
            x_selected = x_selected.to(cur_device)
        x_selected = self.processRange(x_selected.long(), **pr_args)
        x_selected = self.pooling(x_selected)
        x_selected = x_selected.view(x_selected.size(0), -1)

        return x_selected


## MalConv

In [9]:
class MalConv(LowMemConvBase):

    def __init__(self, out_size=2, channels=128, window_size=512, stride=512, embd_size=8, log_stride=None):
        super(MalConv, self).__init__()
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        if not log_stride is None:
            stride = 2**log_stride

        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)


        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)


    def processRange(self, x):
        x = self.embd(x)
        x = torch.transpose(x,-1,-2)

        cnn_value = self.conv_1(x)
        gating_weight = torch.sigmoid(self.conv_2(x))

        x = cnn_value * gating_weight

        return x

    def forward(self, x):
        post_conv = x = self.seq2fix(x)

        penult = x = F.relu(self.fc_1(x))
        x = self.fc_2(x)

        return torch.sigmoid(x)


class MalConvML(LowMemConvBase):

    def __init__(self, out_size=2, channels=128, window_size=512, stride=512, layers=1, embd_size=8, log_stride=None):
        super(MalConvML, self).__init__()
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        if not log_stride is None:
            stride = 2**log_stride

        self.convs = nn.ModuleList([nn.Conv1d(embd_size, channels*2, window_size, stride=stride, bias=True)] + [nn.Conv1d(channels, channels*2, window_size, stride=1, bias=True) for i in range(layers-1)])
        #one-by-one cons to perform information sharing
        self.convs_1 = nn.ModuleList([nn.Conv1d(channels, channels, 1, bias=True) for i in range(layers)])


        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)


    def processRange(self, x):
        x = self.embd(x)
        #x = torch.transpose(x,-1,-2)
        x = x.permute(0,2,1).contiguous()

        for conv_glu, conv_share in zip(self.convs, self.convs_1):
            x = F.leaky_relu(conv_share(F.glu(conv_glu(x.contiguous()), dim=1)))

        return x

    def forward(self, x):
        post_conv = x = self.seq2fix(x)

        penult = x = F.relu(self.fc_1(x))
        x = self.fc_2(x)

        return x, penult, post_conv

## MalConv2

In [10]:
"""
Classifying Sequences of Extreme Length with Constant Memory Applied to Malware Detection
Edward Raff, William Fleshman, Richard Zak, Hyrum Anderson and Bobby Filar and Mark Mclean
https://arxiv.org/abs/2012.09390

Taken from https://github.com/NeuromorphicComputationResearchProgram/MalConv2
"""
import torch
import torch.nn as nn
import torch.nn.functional as F

class MalConv2(LowMemConvBase):

    def __init__(self, out_size=2, channels=128, window_size=512, stride=512, embd_size=8, log_stride=None):
        super(MalConv2, self).__init__()
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        if not log_stride is None:
            stride = 2**log_stride

        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=stride, bias=True)

        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)


    def processRange(self, x):
        x = self.embd(x)
        x = torch.transpose(x,-1,-2)

        cnn_value = self.conv_1(x)
        gating_weight = torch.sigmoid(self.conv_2(x))

        x = cnn_value * gating_weight

        return x

    def forward(self, x):
        post_conv = x = self.seq2fix(x)

        penult = x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return torch.sigmoid(x)

## MalConvGCG

In [11]:
"""
Classifying Sequences of Extreme Length with Constant Memory Applied to Malware Detection
Edward Raff, William Fleshman, Richard Zak, Hyrum Anderson and Bobby Filar and Mark Mclean
https://arxiv.org/abs/2012.09390
"""
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint

class MalConvGCG(LowMemConvBase):

    def __init__(self, out_size=2, channels=128, window_size=512, stride=512, layers=1, embd_size=8, log_stride=None, low_mem=True):
        super(MalConvGCG, self).__init__()
        self.low_mem = low_mem
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        if not log_stride is None:
            stride = 2**log_stride

        self.context_net = MalConvML(out_size=channels, channels=channels, window_size=window_size, stride=stride, layers=layers, embd_size=embd_size)
        self.convs = nn.ModuleList([nn.Conv1d(embd_size, channels*2, window_size, stride=stride, bias=True)] + [nn.Conv1d(channels, channels*2, window_size, stride=1, bias=True) for i in range(layers-1)])

        #These two objs are not used. They were originally present before the F.glu function existed, and then were accidently left in when we switched over. So the state file provided has unusued states in it. They are left in this definition so that there are no issues loading the file that MalConv was trained on.
        #If you are going to train from scratch, you can delete these two lines.
        #self.convs_1 = nn.ModuleList([nn.Conv1d(channels*2, channels, 1, bias=True) for i in range(layers)])
        #self.convs_atn = nn.ModuleList([nn.Conv1d(channels*2, channels, 1, bias=True) for i in range(layers)])

        self.linear_atn = nn.ModuleList([nn.Linear(channels, channels) for i in range(layers)])

        #one-by-one cons to perform information sharing
        self.convs_share = nn.ModuleList([nn.Conv1d(channels, channels, 1, bias=True) for i in range(layers)])


        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)


    #Over-write the determinRF call to use the base context_net to detemrin RF. We should have the same totla RF, and this will simplify logic significantly.
    def determinRF(self):
        return self.context_net.determinRF()

    def processRange(self, x, gct=None):
        if gct is None:
            raise Exception("No Global Context Given")

        x = self.embd(x)
        #x = torch.transpose(x,-1,-2)
        x = x.permute(0,2,1)

        for conv_glu, linear_cntx, conv_share in zip(self.convs, self.linear_atn, self.convs_share):
            x = F.glu(conv_glu(x), dim=1)
            x = F.leaky_relu(conv_share(x))
            x_len = x.shape[2]
            B = x.shape[0]
            C = x.shape[1]

            sqrt_dim = np.sqrt(x.shape[1])
            #we are going to need a version of GCT with a time dimension, which we will adapt as needed to the right length
            ctnx = torch.tanh(linear_cntx(gct))

            #Size is (B, C), but we need (B, C, 1) to use as a 1d conv filter
            ctnx = torch.unsqueeze(ctnx, dim=2)
            #roll the batches into the channels
            x_tmp = x.view(1,B*C,-1)
            #Now we can apply a conv with B groups, so that each batch gets its own context applied only to what was needed
            x_tmp = F.conv1d(x_tmp, ctnx, groups=B)
            #x_tmp will have a shape of (1, B, L), now we just need to re-order the data back to (B, 1, L)
            x_gates = x_tmp.view(B, 1, -1)

            #Now we effectively apply σ(x_t^T tanh(W c))
            gates = torch.sigmoid( x_gates )
            x = x * gates

        return x

    def forward(self, x):

        if self.low_mem:
            global_context = checkpoint.CheckpointFunction.apply(self.context_net.seq2fix,1, x)
        else:
            global_context = self.context_net.seq2fix(x)

        post_conv = x = self.seq2fix(x, pr_args={'gct':global_context})

        penult = x = F.leaky_relu(self.fc_1( x ))
        x = self.fc_2(x)

        return torch.sigmoid(x)

## AvastConv

In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

def vec_bin_array(arr, m=8):
    """
    Arguments:
    arr: Numpy array of positive integers
    m: Number of bits of each integer to retain

    Returns a copy of arr with every element replaced with a bit vector.
    Bits encoded as int8's.
    """
    to_str_func = np.vectorize(lambda x: np.binary_repr(x).zfill(m))
    strs = to_str_func(arr)
    ret = np.zeros(list(arr.shape) + [m], dtype=np.int8)
    for bit_ix in range(0, m):
        fetch_bit_func = np.vectorize(lambda x: x[bit_ix] == '1')
        ret[...,bit_ix] = fetch_bit_func(strs).astype(np.int8)

    return (ret*2-1).astype(np.float32)/16

class AvastConv(LowMemConvBase):

    def __init__(self, out_size=2, channels=48, window_size=32, stride=4, embd_size=8):
        super(AvastConv, self).__init__()
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)
        for i in range(1, 257):
            self.embd.weight.data[i,:] = torch.tensor(vec_bin_array(np.asarray([i])))
        for param in self.embd.parameters():
             param.requires_grad = False

        self.conv_1 = nn.Conv1d(8, channels, window_size, stride=stride, bias=True)
        self.conv_2 = nn.Conv1d(channels, channels*2, window_size, stride=stride, bias=True)
        self.pool = nn.MaxPool1d(4)
        self.conv_3 = nn.Conv1d(channels*2, channels*3, window_size//2, stride=stride*2, bias=True)
        self.conv_4 = nn.Conv1d(channels*3, channels*4, window_size//2, stride=stride*2, bias=True)

        self.fc_1 = nn.Linear(channels*4, channels*4)
        self.fc_2 = nn.Linear(channels*4, channels*3)
        self.fc_3 = nn.Linear(channels*3, channels*2)
        self.fc_4 = nn.Linear(channels*2, out_size)


    def processRange(self, x):
        # Fixed embedding
        with torch.no_grad():
            x = self.embd(x)
            x = torch.transpose(x,-1,-2)

        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool(x)
        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))

        return x

    def forward(self, x):
        post_conv = x = self.seq2fix(x)

        x = F.selu(self.fc_1(x))
        x = F.selu(self.fc_2(x))
        penult = x = F.selu(self.fc_3(x))
        x = self.fc_4(x)

        return torch.sigmoid(x)

# Extensive evaluation of model's robustness

## Model import

In [ ]:
# Loading best models folder

id = '1X8oAi1E183wHkXh4K1I8VdkefBAMS7GM'
gdown.download_folder(id=id)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MalConv2(out_size=1, channels=128, window_size=500, stride=500, embd_size=8, log_stride=None)
model.load_state_dict(torch.load("/content/drive/Shareddrives/AFC/models/malconv2_model.pt"), strict=False)
model = model.to(device)

model_wrapper = MalConvWrapper(model, max_len=2 ** 20)

## Selection of malwares to obfuscate

In [ ]:
import shutil, os

N_SAMPLES = 100
malware_directory = '/content/test/malware'
malware_dest_dir = '/content/malware_to_obfuscate'

classification_results = 0
if not os.path.exists(malware_dest_dir):
  os.mkdir(malware_dest_dir)

# search for a file classified as malware (with classification result > 0.5)
while len(os.listdir(os.path.join(malware_dest_dir))) < N_SAMPLES:
  # input sample that we want to obfuscate
  filename = random.choice( os.listdir(malware_directory) )
  path = os.path.join(malware_directory,  filename)
  with open(path, "rb") as file_handle:
      code = file_handle.read()
      # read the executable as numpy array
      x = np.frombuffer(code, dtype=np.uint8)
  classification_results = model_wrapper.classify_sample(x)

  if classification_results > 0.5:
    shutil.copyfile(path, os.path.join(malware_dest_dir, filename))

print(len(os.listdir(os.path.join("/content/malware_to_obfuscate"))))

We can do a sanity check to verify that malware and benign are correcly classified before the attack.

In [ ]:
# Verify if benign are correctly classified
benign_path = '/content/test/benign'
flag = False
for f in os.listdir(benign_path):
    with open(os.path.join(benign_path, f), 'rb') as file:
        file_bytes = file.read()
        if model_wrapper.classify_sample(file_bytes) >= 0.5:
            print("Benign misclassified: ", f)
            os.remove(os.path.join(benign_path, f))
            if not os.path.exists(os.path.join(benign_path, f)):
                print("File removed")

# Verify if malware are correctly
malware_path = '/content/malware_to_obfuscate'
malware_files = os.listdir(malware_path)
for f in malware_files:
    with open(os.path.join(malware_path, f), 'rb') as file:
        file_bytes = file.read()
        if model_wrapper.classify_sample(file_bytes) < 0.5:
            print("Malware misclassified: ", f)

## GAMMA attack

### Selecting attack parameters

In [ ]:
# Section population with how_many = 25
section_population_25, _ = create_section_population_from_folder(
    benign_path, how_many = 25, sections_to_extract=['.data','.rdata', '.idata', '.rodata'],
    cache_file='/content/section_population.pkl')

# Section population with how_many = 50
section_population_50, _ = create_section_population_from_folder(
    benign_path, how_many = 50, sections_to_extract=['.data','.rdata', '.idata', '.rodata'],
    cache_file='/content/section_population.pkl')
print("Section extracted")

# Attack parameters
# lambda_values = [10e-3, 10e-5, 10e-7, 10e-9]
lambda_values = [10e-3]
query_values = [20]
POPULATION_SIZE = 20

### Defining GAMMA attack function

In [27]:
from ml_pentest.attacks.blackbox.genetic_attack.GAMMA.attack_utils import generate_adv_samples_from_folder

def gamma_attack(base_path, section_population):
  if not os.path.exists(os.path.join(base_path, 'samples')):
      os.makedirs(os.path.join(base_path, 'samples'))
  if not os.path.exists(os.path.join(base_path, 'results')):
      os.makedirs(os.path.join(base_path, 'results'))

  for lambda_value in lambda_values:
      if not os.path.isdir(os.path.join(base_path, 'samples',str(lambda_value))):
          os.mkdir(os.path.join(base_path, 'samples',str(lambda_value)))
      for query_budget in query_values:
          destination_folder = os.path.join(base_path, 'samples', str(lambda_value),str(query_budget))
          if not os.path.isdir(destination_folder):
              os.mkdir(destination_folder)
          print("Lambda: ", lambda_value, "Query budget: ", query_budget)
          attack = GammaSectionInjection(section_population=section_population, model_wrapper=model_wrapper,
                                      population_size=POPULATION_SIZE, lambda_value=lambda_value, iterations=100,
                                      debug=False, hard_label=False, query_budget=query_budget,
                                      stagnation=5)
          result_file = os.path.join(base_path, 'results', 'results_'+str(lambda_value)+'_'+str(query_budget)+'.json')
          generate_adv_samples_from_folder(source_folder=malware_path,
                                        destination_folder=destination_folder, gamma_attack=attack, model=model_wrapper,result_file=result_file)

In [ ]:
#GAMMA attack with how_many = 25
gamma_attack('/content/attack_results_25', section_population_25)

#GAMMA attack with how_many = 50
gamma_attack('/content/attack_results_50', section_population_50)

## Evaluate the attack

In [24]:
base_path = '/content/attack_results_25/results'
variant_path = '/content/attack_results_50/results'

models = dict()
models["how_many = 25"] = base_path
models["how_many = 50"] = variant_path

In [ ]:
from ml_pentest.attack_reports.blackbox.genetic_attack.GAMMA.analize_results import compute_mean_times, print_gamma_results, plot_detection_rate, plot_injected_bytes, plot_heatmap, plot_detection_rate_vs_query_budget, plot_avg_injected_bytes_vs_query_budget, plot_gamma_attack
import os
import numpy as np

malware_files = os.listdir(malware_path)
class_results = []

for f in malware_files:
    with open(os.path.join(malware_path, f), 'rb') as file:
        file_bytes = file.read()
        class_results.append( model_wrapper.classify_sample(file_bytes) )

avg_detection_rate = np.mean(class_results)

model_name='MalConv2'

print("============================ GAMMA Results ============================")

print("===== how_many = 25 =====")
for lambda_value in lambda_values:
    print_gamma_results(base_path, lambda_value, query_values, avg_detection_rate = avg_detection_rate)

print("===== how_many = 50 =====")
for lambda_value in lambda_values:
    print_gamma_results(variant_path, lambda_value, query_values, avg_detection_rate = avg_detection_rate)


save_path = '/content/attack_results/plots'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Plotting graphs for how_many = 25
plot_detection_rate(base_path, os.path.join(base_path, '25'), query_values, lambda_values, 'detection_rate_gamma.png', model_name=model_name)
plot_injected_bytes(base_path, os.path.join(base_path, '25'), query_values, lambda_values, 'GAMMA', 'gamma_injected_bytes.png', model_name=model_name,  x_range_kb=None)
plot_heatmap(base_path, os.path.join(base_path, '25'), query_values, lambda_values, 'heatmap.png')

# Plotting graphs for how_many = 50
plot_detection_rate(base_path, os.path.join(base_path, '50'), query_values, lambda_values, 'detection_rate_gamma.png', model_name=model_name)
plot_injected_bytes(base_path, os.path.join(base_path, '50'), query_values, lambda_values, 'GAMMA', 'gamma_injected_bytes.png', model_name=model_name,  x_range_kb=None)
plot_heatmap(base_path, os.path.join(base_path, '50'), query_values, lambda_values, 'heatmap.png')

# Plotting graphs to compare two attacks
plot_gamma_attack(base_path, variant_path, save_path, query_values, lambda_values, 'gamma_attack.png', model_name=model_name)
plot_detection_rate_vs_query_budget(models, query_values, lambda_values, save_path, 'detection_vs_query.png')
plot_avg_injected_bytes_vs_query_budget(models, query_values, lambda_values, save_path, file_name='avg_injected_bytes_vs_query_budget.png', lower_limit=None, upper_limit=None)
